In [1]:
from surprise import Dataset
from surprise import SVD

import pandas as pd
import re
import pywikibot as pw
import numpy as np
import pickle

Skipping loading of user-config.py.
family and mylang are not set.
Defaulting to family='test' and mylang='test'.


In [2]:
site = pw.Site('en', 'wikipedia')

In [3]:
ml_100k_data = Dataset.load_builtin('ml-100k')
ml_100k_data_frame = pd.DataFrame(ml_100k_data.raw_ratings)

print("users=", len(ml_100k_data_frame[0].unique()))
print("items=", len(ml_100k_data_frame[1].unique()))
print("ratings=", len(ml_100k_data_frame))

users= 943
items= 1682
ratings= 100000


In [4]:
movies = pd.read_csv("ml-100k\\u.item", sep="|", header = None, encoding='latin-1')[[0,1]]
movies = movies.values[:,1]

In [5]:
def get_film_wiki_categories(film_name):
    dic = {}
    names = generate_film_names(film_name)
    for name in names:
        page = pw.Page(site, name)
        occ = get_occ_of_film_in_page(page)
        dic[name] = occ
    max_key = max(dic, key=dic.get)
    real_page = pw.Page(site, max_key)
    return __get_categories_of_wiki_page(real_page)
        

In [6]:
def generate_film_names(film_name):
    names = []
    name_without_year = __remove_year_from_name(film_name)
    names.append(film_name)
    names.append(name_without_year)
    names.append(name_without_year + " (film)")
    
    if ")" in film_name:
        names.append(film_name.replace(")",' film)'))
    
    if ", The" in film_name:
        tmp = "The " + film_name.replace(", The",'').strip()
        names.append(tmp)
        names.append(__remove_year_from_name(tmp))
        names.append(__remove_year_from_name(tmp) + " (film)")
        if ")" in tmp:
            names.append(tmp.replace(")",' film)'))
            
    if ", A" in film_name:
        tmp = "A " + film_name.replace(", A",'').strip()
        names.append(tmp)
        names.append(__remove_year_from_name(tmp))
        names.append(__remove_year_from_name(tmp) + " (film)")
        if ")" in tmp:
            names.append(tmp.replace(")",' film)'))
            
    if ", An" in film_name:
        tmp = "An " + film_name.replace(", An",'').strip()
        names.append(tmp)
        names.append(__remove_year_from_name(tmp))
        names.append(__remove_year_from_name(tmp) + " (film)")
        if ")" in tmp:
            names.append(tmp.replace(")",' film)'))
            
    if "Colors" in film_name:
        names.append(film_name.replace("Colors",'Colours').strip())
        
    if ":" in film_name:
        names.append(film_name.replace(":",''))
        names.append(__remove_year_from_name(film_name.replace(":",'')))
        
    removed_text_in_brackets = __remove_text_in_brackets(film_name)
    names.append(removed_text_in_brackets)
    names.append(__remove_year_from_name(removed_text_in_brackets))
    return names

In [7]:
def get_occ_of_film_in_page(page):
    if page.exists():
        categories = __get_categories_of_wiki_page(page)
        return __count_number_of_film_occ_in_categories(categories)
    return 0

In [8]:
def __remove_text_in_brackets(name):
    m = re.search('\([^0-9]*\)', name)
    if m:
        found = m.group(0)
        return name.replace(found + " ",'').strip()
    return name

In [9]:
def __remove_year_from_name(name):
    m = re.search('(.+)\W(\(\d{4}\))', name)
    if m:
        found = m.group(2)
        return name.replace(found,'').strip()
    return name

In [10]:
def __get_categories_of_wiki_page(page):
    return [i.title() for i in list(page.categories())]

In [11]:
def __count_number_of_film_occ_in_categories(categories):
    counter = 0
    for cat in categories:
        if "film" in cat:
            counter = counter + 1
    return counter

In [12]:
def get_movie_id_by_name(name):
    return df.loc[df[1] == name][0].values[0]

In [13]:
def jaccard_similarity(list1, list2):
    intersection = len(list(set(list1).intersection(list2)))
    union = (len(list1) + len(list2)) - intersection
    if union == 0:
        return 0
    return float(intersection / union)

In [ ]:
dic = {}
for movie in movies:
    categories = get_film_wiki_categories(movie)
    dic[movie] = categories

In [ ]:
with open('movie_categories.pickle', 'wb') as handle:
    pickle.dump(dic, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [182]:
df = pd.read_csv("ml-100k\\u.item", sep="|", header = None, encoding='latin-1')[[0,1]]

In [251]:
item_to_item = []
for key_1, value_1 in dic.items():
    for key_2, value_2 in dic.items():
        id_1 = get_movie_id_by_name(key_1)
        id_2 = get_movie_id_by_name(key_2)
        similarity = jaccard_similarity(value_1,value_2)
        item_to_item.append([id_1,id_2,similarity])

In [252]:
a = np.array(item_to_item)

In [253]:
d = pd.DataFrame(a)

In [255]:
d.to_csv("artificial_ratings.csv", index=False)

In [167]:
100 - (sum([x[1]==0 for x in arr]) /len(arr))*100

85.4934601664685